In [3]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 9.1026 ,random_state=0 , l1_ratio= 0.997, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96187 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_201000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_20_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [8]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=889886, Mon Apr 13 13:31:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.005771724768200e+05 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
    2     26 5.007830628051328e+05 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0
    3     39 4.992893144785305e+05 1.1e+00 1.22e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [502448.33900809] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [501972.72890218] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -1.501285408117396e+05 1.7e+00 6.28e-03  5e-03  6e-03 0:00.9
  200   2600 -1.508080248396361e+05 1.8e+00 7.88e-05  5e-05  7e-05 0:01.8
  300   3900 -1.508081315701073e+05 2.0e+00 1.39e-06  8e-07  1e-06 0:02.7
  400   5200 -1.508081315854873e+05 2.3e+00 1.65e-08  8e-09  1e-08 0:03.6
  500   6500 -1.508081315854891e+05 2.6e+00 1.95e-09  9e-10  1e-09 0:04.5
  600   7800 -1.508081315854891e+05 3.8e+00 1.00e-09  4e-10  6e-10 0:05.4
  700   9100 -1.508081315854890e+05 4.5e+00 1.52e-09  6e-10  9e-10 0:06.3
  800  10400 -1.508081315854890e+05 5.5e+00 4.15e-10  1e-10  3e-10 0:07.2
  900  11700 -1.508081315854891e+05 7.0e+00 1.90e-10  7e-11  1e-10 0:08.1
 1000  13000 -1.508081315854894e+05 7.9e+00 1.48e-10  5e-11  1e-10 0:09.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  14300 -1.508081315854890e+05 9.2e+00 3.32e-10  1e-10  2e-10 0:09.9
 1200  15600 -1.508081315854890e+05 1.1e+01 3.89e-10  1e-10  3e-10 0:10.8
 1300  16900 -1.508081315854890e+05 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [490033.64479747] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [490031.20255471] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 4.808067465421942e+05 1.7e+00 6.63e-03  5e-03  6e-03 0:00.8
  200   2600 4.807940003419762e+05 1.8e+00 1.08e-04  7e-05  9e-05 0:01.5
  300   3900 4.807939976961465e+05 2.0e+00 2.21e-06  1e-06  2e-06 0:02.3
  400   5200 4.807939976955170e+05 2.0e+00 3.01e-08  1e-08  2e-08 0:03.0


...

  100   1300 4.808098417618808e+05 1.9e+00 7.54e-03  6e-03  8e-03 0:00.7
  200   2600 4.807940009511980e+05 1.8e+00 1.33e-04  9e-05  1e-04 0:01.5
  300   3900 4.807939976959170e+05 2.0e+00 1.83e-06  1e-06  1e-06 0:02.2
  400   5200 4.807939976955170e+05 2.0e+00 3.06e-08  1e-08  2e-08 0:02.9
  489   6357 4.807939976955169e+05 2.5e+00 4.30e-09  2e-09  3e-09 0:03.6


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=480793.997696, sigma=4.97e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=475)
  ')')


termination on tolfunhist=1e-12 (Mon Apr 13 13:43:00 2020)
final/bestever f-value = 4.807940e+05 4.807940e+05
incumbent solution: [ 0.07814174 -0.05789998 -0.04363781 -0.23399829 -0.14717545  0.06621662
  0.1218278   0.07358866 ...]
std deviations: [2.44026715e-09 2.09684867e-09 2.15714621e-09 2.33461388e-09
 2.21330208e-09 2.15466914e-09 2.43908803e-09 2.62706548e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=828814, Mon Apr 13 13:43:01 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.514145198879859e+05 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 5.298475848093706e+05 1.1e+00 1.18e-01  1e-01  1e-01 0:00.0
    3     39 5.115407167857056e+05 1.1e+00 1.18e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [551414.51988799] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [571307.89626499] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   77   1001 -1.088785445454183e+11 1.7e+00 1.31e+02  1e+02  1e+02 0:00.8
termination on tolfacupx=1000.0 (Mon Apr 13 13:43:02 2020)
final/bestever f-value = -9.876514e+10 -1.088785e+11
incumbent solution: [ 227.36426834 -336.20205424 -270.46700252 -721.45442951 -101.85414666
   -8.38701251  268.19174508 -143.64954968 ...]
std deviations: [116.16543697 117.84795731 117.72593425 126.36096248 126.97037677
 122.68836926 118.3242245  133.59019961 ...]


In [9]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F20_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
KNN_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:25])
    SVM_Fun [i] = F20(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F20(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([25079.42300849]), count=array([1])) 7.708029148314079e-05
SVM
ModeResult(mode=array([25501.00190677]), count=array([1])) 0.00026854971530911335
ELN
ModeResult(mode=array([3.31565892e+09]), count=array([1])) 1073294883.9541894
